In [ ]:
import hicstraw
import math
import numpy as np
from sklearn.decomposition import NMF
from scipy.stats import pearsonr
import h5py
import hicstraw
import numpy as np
from sklearn.decomposition import NMF
from scipy.stats import pearsonr
import h5py
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from sklearn.decomposition import PCA
import scipy.io
from sklearn.manifold import TSNE
import gzip
from sklearn.decomposition import NMF
import tensorly as tl
from tensorly.decomposition import non_negative_parafac
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
from numpy.linalg import norm
from sklearn.mixture import GaussianMixture
import warnings
from scipy.stats import zscore
from scipy.interpolate import interp1d, interp2d
import os
import sys
from scipy.sparse import coo_matrix, csr_matrix, triu, tril
import os
from collections import defaultdict
import random
import glob
import pickle
import seaborn as sns
import numpy as np
import h5py
import os

base_directory = "/home/dwk681/workspace/multi_omics_hic_clustering/scNOMeHiC_20210127/250Kb/methylation_files/"
file_path = base_directory + 'b37.autosome.250kb_interval.add_value.methy.GM_IMR90_128_samples.bed.gz'

def normalize_matrix_columns(A):
    """
    Normalize each column of the matrix so that each has a norm of one.
    
    Parameters:
    - A: a 2D NumPy array (matrix)
    
    Returns:
    - normalized_A: a matrix where each column of A has been divided by its L2 norm
    """
    # Calculate the L2 norm for each column
    column_norms = np.linalg.norm(A, axis=0)
    
    # Avoid division by zero
    if np.any(column_norms == 0):
        raise ValueError("One or more columns have zero norm. Cannot normalize those columns.")
    
    # Normalize each column by its norm
    normalized_A = A / column_norms
    
    return normalized_A

def calculate_matrix(file_path):
    # Initialize an empty list to store the data
    data = []

    # Read the compressed file using gzip
    with gzip.open(file_path, 'rt') as file:
        for line in file:
            # Split the line into fields
            fields = line.strip().split('\t')

            # Extract relevant entries for calculations
            numerator_indices = range(6, len(fields), 2)
            denominator_indices = range(7, len(fields), 2)

            row_data = []
            for num_idx, denom_idx in zip(numerator_indices, denominator_indices):
                numerator = float(fields[num_idx])
                denominator = float(fields[denom_idx])

                # Check if denominator is zero, set entry to 0, else perform the division
                row_data.append(0 if denominator == 0 else numerator / denominator)

            data.append(row_data)

    # Convert the list of lists into a NumPy array
    data_matrix = np.array(data)

    return data_matrix

methylation_matrix_1Mb = normalize_matrix_columns(calculate_matrix(file_path))
print(methylation_matrix_1Mb.shape)


In [13]:
import os
import pyBigWig
import numpy as np
from scipy.stats import ks_2samp
import h5py

# Import variables from config_and_print.py
from config_and_print import methy_directory, output_directory, resolutions, filtered_list

# Read the filtered bam list
with open(filtered_list, 'r') as f:
    filtered_prefixes = set(line.strip() for line in f)

# Function to process each bigWig file
def process_bigwig_file(bigwigfile, large_bin_size, small_bin_size, resolution_label, base_directory, prefix):
    bw = pyBigWig.open(bigwigfile)
    chromosomes = bw.chroms()

    for chrom in chromosomes:
        # Ensure chrom_string is always in the format 'chrN'
        if isinstance(chrom, int) or chrom.isdigit():
            chrom_string = f'chr{chrom}'
        else:
            chrom_string = chrom if chrom.startswith('chr') else f'chr{chrom}'

        if chrom_string in [f'chr{i}' for i in range(1, 23)]:
            # Create chromosome-specific directory if it doesn't exist
            chrom_directory = os.path.join(base_directory, chrom_string)
            os.makedirs(chrom_directory, exist_ok=True)

            # Define the filename for the matrix
            new_filename = os.path.join(chrom_directory, f'{prefix}_{chrom_string}.h5')

            # Check if the file already exists
            if os.path.exists(new_filename):
                print(f"File {new_filename} already exists, skipping computation.")
                continue

            chrom_length = chromosomes[chrom]
            large_bins = range(0, chrom_length, large_bin_size)
            num_large_bins = len(large_bins)

            integrals = np.zeros((large_bin_size // small_bin_size, num_large_bins))

            for i, large_bin_start in enumerate(large_bins):
                for j in range(large_bin_size // small_bin_size):
                    small_bin_start = large_bin_start + j * small_bin_size
                    small_bin_end = small_bin_start + small_bin_size

                    # Ensure that the coordinates are within the chromosome boundaries
                    if small_bin_start < 0:
                        small_bin_start = 0
                    if small_bin_end > chrom_length:
                        small_bin_end = chrom_length

                    # Check if the adjusted small_bin_start is still greater than or equal to small_bin_end
                    if small_bin_start >= small_bin_end:
                        continue

                    try:
                        # Get data within the small bin
                        small_bin_data = bw.values(chrom, int(small_bin_start), int(small_bin_end))

                        # Calculate the integral of data in the small bin
                        integral = np.trapz(small_bin_data)

                        # Store the integral in the corresponding list
                        integrals[j, i] = integral
                    except Exception as e:
                        # Print the exception and relevant information
                        print(f"An error occurred for chromosome {chrom}, large bin {i}, small bin {j}: {e}")

            # create your matrices
            num_rows, num_columns = integrals.shape
            p_value_ks_matrix = np.zeros((num_columns, num_columns))
            epsilon = 1e-10  # Define your epsilon value here

            for i in range(num_columns):
                for j in range(i, num_columns):
                    if i == j:
                        p_value_ks_matrix[i, j] = 1.0
                    else:
                        _, ks_p_value = ks_2samp(integrals[:, i], integrals[:, j])

                        p_value_ks_matrix[i, j] = ks_p_value
                        p_value_ks_matrix[j, i] = ks_p_value
                        
                        #p_value_ks_matrix[i, j] = np.log10(ks_p_value + epsilon)
                        #p_value_ks_matrix[j, i] = np.log10(ks_p_value + epsilon)

            # take the correlation matrices of all the matrices
            correlation_ks_p_value = np.corrcoef(p_value_ks_matrix) + 1

            # Save the correlation epigenetic matrix as a .h5 file
            with h5py.File(new_filename, 'w') as hf:
                hf.create_dataset('methy_matrix', data=p_value_ks_matrix)

# Ensure resolutions is processed as a list of strings
resolutions_list = [res.strip() for res in resolutions.split(',')]

# Iterate through the files in methy_directory
for root, dirs, files in os.walk(methy_directory):
    for file in files:
        if file.endswith('.methy.b37.bw'):
            # Extract the prefix for filtering
            prefix = '.'.join(file.split('.')[:2])
            if prefix in filtered_prefixes:
                bigwigfile = os.path.join(root, file)
                print(f"Processing {bigwigfile}")
                for resolution in resolutions_list:
                    try:
                        res_value, res_label = resolution.split(":")
                        # Create the base directory for the matrices
                        base_directory = os.path.join(output_directory, f'methy_{res_label}_correlation_dir')
                        os.makedirs(base_directory, exist_ok=True)
                        large_bin_size = int(res_value)
                        small_bin_size = large_bin_size // 100
                        process_bigwig_file(bigwigfile, large_bin_size, small_bin_size, res_label, base_directory, prefix)
                    except ValueError:
                        print(f"Invalid resolution format: {resolution}")


Processing /home/dwk681/workspace/cluster_cells_from_GSE189158_NOMe_HiC/filesFromCluster/bam/methylation/filter_low_qual/sc24.ACTTGA.methy.b37.bw
Processing /home/dwk681/workspace/cluster_cells_from_GSE189158_NOMe_HiC/filesFromCluster/bam/methylation/filter_low_qual/sc14.TAGCTT.methy.b37.bw
Processing /home/dwk681/workspace/cluster_cells_from_GSE189158_NOMe_HiC/filesFromCluster/bam/methylation/filter_low_qual/sc17.CGATGT.methy.b37.bw
Processing /home/dwk681/workspace/cluster_cells_from_GSE189158_NOMe_HiC/filesFromCluster/bam/methylation/filter_low_qual/sc19.ACTTGA.methy.b37.bw


KeyboardInterrupt: 

['1000000', '1Mb']